In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics supervision opencv-python gdown

In [4]:
# Download and unzip pigs-tracking.zip
!gdown https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD -O /content/pigs-tracking.zip
!unzip /content/pigs-tracking.zip -d /content/videos

# Download and unzip pigs-yolov8-weights.zip
!gdown https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5- -O /content/pigs-yolov8-weights.zip
!unzip /content/pigs-yolov8-weights.zip -d /content

Downloading...
From (original): https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD
From (redirected): https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD&confirm=t&uuid=154774e1-6732-440c-bed0-7b6e347ced4a
To: /content/pigs-tracking.zip
100% 1.18G/1.18G [00:16<00:00, 72.9MB/s]
Archive:  /content/pigs-tracking.zip
  inflating: /content/videos/Test Vid 1.mp4  
  inflating: /content/videos/Test Vid 2.mp4  
  inflating: /content/videos/Test Vid 3.mp4  
  inflating: /content/videos/Test Vid 5.mp4  
  inflating: /content/videos/Test Vid 8.mp4  
  inflating: /content/videos/Test Vid 6.mp4  
  inflating: /content/videos/Test Vid 7.mp4  
Downloading...
From (original): https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5-
From (redirected): https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5-&confirm=t&uuid=52b626c3-24bb-4e64-8f3c-30db62992cf8
To: /content/pigs-yolov8-weights.zip
100% 169M/169M [00:02<00:00, 61.9MB/s]
Archive:  /content/pigs-yol

In [9]:
import os
from ultralytics import YOLO
import supervision as sv
import cv2

video_dir = '/content/videos' # This variable is now less critical for defining input, but still used for context
weights_path = '/content/v9/weights/best.pt'

model = YOLO(weights_path)

tracker = sv.ByteTrack()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

output_dir = '/content/tracked_videos'
os.makedirs(output_dir, exist_ok=True)

# --- MODIFICATION STARTS HERE ---
video_files = [
    "/content/videos/Test Vid 1.mp4",
    # "/content/videos/Test Vid 2.mp4",
    # "/content/videos/Test Vid 3.mp4",
    # "/content/videos/Test Vid 5.mp4",
    # "/content/videos/Test Vid 6.mp4",
    # "/content/videos/Test Vid 7.mp4",
    # "/content/videos/Test Vid 8.mp4"
]
# --- MODIFICATION ENDS HERE ---

for video_path in video_files:

    print(f"Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        continue

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    output_video_path = os.path.join(output_dir, f"tracked_{os.path.basename(video_path)}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = tracker.update_with_detections(detections=detections)

        labels = [
            f"pig {tracker_id}"
            for tracker_id in detections.tracker_id if tracker_id is not None
        ]

        annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
        annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)

        out.write(annotated_frame)

        frame_count += 1
        if frame_count % 200 == 0:
            print(f"    Processed {frame_count} frames for {os.path.basename(video_path)}")

    cap.release()
    out.release()
    print(f"Finished processing {video_path}. Output saved to {output_video_path}")

print("All videos processed!")

Processing video: /content/videos/Test Vid 1.mp4
    Processed 200 frames for Test Vid 1.mp4
    Processed 400 frames for Test Vid 1.mp4
    Processed 600 frames for Test Vid 1.mp4
    Processed 800 frames for Test Vid 1.mp4
    Processed 1000 frames for Test Vid 1.mp4
    Processed 1200 frames for Test Vid 1.mp4
    Processed 1400 frames for Test Vid 1.mp4
Finished processing /content/videos/Test Vid 1.mp4. Output saved to /content/tracked_videos/tracked_Test Vid 1.mp4
All videos processed!


In [13]:
import os
import cv2
from IPython.display import Video, display

# Directory where your processed MP4 videos are saved
input_mp4_dir = '/content/tracked_videos'
output_webm_dir = '/content/converted_webm_videos' # A dedicated directory for WebM outputs

os.makedirs(output_webm_dir, exist_ok=True) # Ensure the output directory exists

# Get a list of all MP4 files in the input directory
mp4_files_to_convert = [
    os.path.join(input_mp4_dir, f)
    for f in os.listdir(input_mp4_dir)
    if f.endswith('.mp4')
]

if not mp4_files_to_convert:
    print(f"No MP4 files found in {input_mp4_dir} to convert.")
else:
    print(f"Found {len(mp4_files_to_convert)} MP4 files to convert.")
    for found_mp4_path in mp4_files_to_convert:
        if not os.path.exists(found_mp4_path):
            print(f"Error: Input MP4 file not found at {found_mp4_path} - Skipping.")
            continue

        base_name = os.path.splitext(os.path.basename(found_mp4_path))[0]
        output_webm_path = os.path.join(output_webm_dir, f"{base_name}.webm")

        print(f"Converting '{os.path.basename(found_mp4_path)}' to WebM...")

        # --- MODIFIED FFmpeg COMMAND ---
        # -vf scale=960:-2 : Scales resolution to 960x540 (half of 1920x1080)
        # -b:v 500k       : Sets video bitrate to 500 kb/s (half of 1M)
        # -threads 8      : Uses 8 threads for encoding
        # -cpu-used 7     : Aggressive speed preset for VP9 (0=best quality/slowest, 8=lowest quality/fastest)
        !ffmpeg -i "{found_mp4_path}" -vcodec vp9 -acodec libvorbis -b:v 500k -threads 8 -deadline realtime -vf scale=960:-2 -cpu-used 7 "{output_webm_path}" -y
        # --- END MODIFIED COMMAND ---

        if os.path.exists(output_webm_path):
            print(f"Conversion complete for {os.path.basename(found_mp4_path)}. Saved to: {output_webm_path}")
            # Display each converted video
            print(f"Displaying {os.path.basename(output_webm_path)}:")
            display(Video(output_webm_path, embed=True, html_attributes="controls autoplay loop"))
        else:
            print(f"Error: WebM file not found after conversion for {os.path.basename(found_mp4_path)}. Cannot display.")

    print("\nAll specified MP4 videos have been processed for WebM conversion.")

Output hidden; open in https://colab.research.google.com to view.

In [14]:
!cp -r /content/tracked_videos /content/drive/MyDrive/Data
!cp -r /content/converted_webm_videos /content/drive/MyDrive/Data